In [ ]:
#pip install langdetect

In [63]:
# Import dependencies
import pandas as pd
import numpy as np
import re
from path import Path
from langdetect import detect, detect_langs

In [64]:
# Create DataFrame from CSV file
music_df = pd.read_csv('../preliminary_dataframes/nlp_df.csv')
music_df.head()

,Unnamed: 0,song,artist,category,lyrics,words,filtered,hashedValues,features
0,0,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,34,You put me on a pedestal and tell me Im the b...,"['', 'you', 'put', 'me', 'on', 'a', 'pedestal'...","['', 'put', 'pedestal', 'tell', 'im', 'best', ...","(262144,[3386,3924,18184,21823,30548,31015,375...","(262144,[3386,3924,18184,21823,30548,31015,375..."
1,1,positions,Ariana Grande,34,Heaven sent you to me Im just hopin I dont re...,"['', 'heaven', 'sent', 'you', 'to', 'me', 'im'...","['', 'heaven', 'sent', 'im', 'hopin', 'dont', ...","(262144,[7231,21823,31015,39504,51471,55334,64...","(262144,[7231,21823,31015,39504,51471,55334,64..."
2,2,Therefore I Am,Billie Eilish,34,Im not your friend Or anything damn You think...,"['', 'im', 'not', 'your', 'friend', 'or', 'any...","['', 'im', 'friend', 'anything', 'damn', 'thin...","(262144,[18176,22346,23071,23087,24175,31015,3...","(262144,[18176,22346,23071,23087,24175,31015,3..."
3,3,Levitating (feat. DaBaby),Dua Lipa,34,Billboard Baby Dua Lipa los hace bailar cuand...,"['', 'billboard', 'baby', 'dua', 'lipa', 'los'...","['', 'billboard', 'baby', 'dua', 'lipa', 'los'...","(262144,[522,764,1212,2298,7188,7913,7975,8884...","(262144,[522,764,1212,2298,7188,7913,7975,8884..."
4,4,Dakiti,Bad Bunny,34,Baby ya yo me enteré se nota cuando me ve Ahí...,"['', 'baby', 'ya', 'yo', 'me', 'enteré', 'se',...","['', 'baby', 'ya', 'yo', 'enteré', 'se', 'nota...","(262144,[764,1781,2409,3165,5555,6523,9192,125...","(262144,[764,1781,2409,3165,5555,6523,9192,125..."


In [65]:
# Try to remove songs with spanish lyrics
for index, row in music_df.iterrows():
    if ('é' in row['lyrics']) or ('í' in row['lyrics']):
        music_df = music_df.drop(index)
music_df.head()

,Unnamed: 0,song,artist,category,lyrics,words,filtered,hashedValues,features
0,0,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,34,You put me on a pedestal and tell me Im the b...,"['', 'you', 'put', 'me', 'on', 'a', 'pedestal'...","['', 'put', 'pedestal', 'tell', 'im', 'best', ...","(262144,[3386,3924,18184,21823,30548,31015,375...","(262144,[3386,3924,18184,21823,30548,31015,375..."
1,1,positions,Ariana Grande,34,Heaven sent you to me Im just hopin I dont re...,"['', 'heaven', 'sent', 'you', 'to', 'me', 'im'...","['', 'heaven', 'sent', 'im', 'hopin', 'dont', ...","(262144,[7231,21823,31015,39504,51471,55334,64...","(262144,[7231,21823,31015,39504,51471,55334,64..."
2,2,Therefore I Am,Billie Eilish,34,Im not your friend Or anything damn You think...,"['', 'im', 'not', 'your', 'friend', 'or', 'any...","['', 'im', 'friend', 'anything', 'damn', 'thin...","(262144,[18176,22346,23071,23087,24175,31015,3...","(262144,[18176,22346,23071,23087,24175,31015,3..."
5,5,Errbody,Lil Baby,34,Flyer than everybody Section 8 just straight ...,"['', 'flyer', 'than', 'everybody', 'section', ...","['', 'flyer', 'everybody', 'section', '8', 'st...","(262144,[1303,1398,1745,3280,4372,4978,6715,82...","(262144,[1303,1398,1745,3280,4372,4978,6715,82..."
6,6,Whoopty,CJ,34,Loyalty over royalty yall niggas know the vib...,"['', 'loyalty', 'over', 'royalty', 'yall', 'ni...","['', 'loyalty', 'royalty', 'yall', 'niggas', '...","(262144,[1689,3186,6116,6524,8538,10345,11209,...","(262144,[1689,3186,6116,6524,8538,10345,11209,..."


In [66]:
# Create a list of all words, languages, and filtered words
words_list = []
languages = []
filtered_words_list = []
for index, row in music_df.iterrows():
    filtered_words = music_df['filtered'][index]
    filtered_words = filtered_words.replace(',', '').replace('.', '').replace('`', '')
    filtered_words = filtered_words.replace('[', '').replace(']', '').replace('{', '').replace('}', '')
    filtered_words = filtered_words.replace('!', '').replace("'", '').replace('"', '').replace('”', '')
    filtered_words = filtered_words.replace(':', '').replace(';', '').replace('*', '')
    filtered_words = filtered_words.replace('“', '').replace('_', '').replace('—', '').replace('–', ' ')
    filtered_words = filtered_words.replace('\\u2005', ' ').replace('¨', '')
    filtered_words = filtered_words.replace('‘', '')
    filtered_words = filtered_words.replace('=', ' ').replace('·', '')
    while ('\\u200e' in filtered_words) or ('\\xa0' in filtered_words):
        filtered_words = filtered_words.replace('\\u200e', '')
        filtered_words = filtered_words.replace('\\xa0', '')
    filtered_words_list.append(filtered_words)
    try:
        lang = detect_langs(filtered_words)
        languages.append(str(lang[0])[0:2])
    except:
        languages.append(np.nan)
    unique_words = list(set(filtered_words.strip().split(' ')))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
len(word_columns)


70134

In [67]:
# Add a language column to the DataFrame and replace the filtered column
music_df['language'] = languages
music_df['filtered'] = filtered_words_list
music_df.head()

,Unnamed: 0,song,artist,category,lyrics,words,filtered,hashedValues,features,language
0,0,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,34,You put me on a pedestal and tell me Im the b...,"['', 'you', 'put', 'me', 'on', 'a', 'pedestal'...",put pedestal tell im best raise sky im short ...,"(262144,[3386,3924,18184,21823,30548,31015,375...","(262144,[3386,3924,18184,21823,30548,31015,375...",en
1,1,positions,Ariana Grande,34,Heaven sent you to me Im just hopin I dont re...,"['', 'heaven', 'sent', 'you', 'to', 'me', 'im'...",heaven sent im hopin dont repeat history boy ...,"(262144,[7231,21823,31015,39504,51471,55334,64...","(262144,[7231,21823,31015,39504,51471,55334,64...",en
2,2,Therefore I Am,Billie Eilish,34,Im not your friend Or anything damn You think...,"['', 'im', 'not', 'your', 'friend', 'or', 'any...",im friend anything damn think youre man think...,"(262144,[18176,22346,23071,23087,24175,31015,3...","(262144,[18176,22346,23071,23087,24175,31015,3...",en
5,5,Errbody,Lil Baby,34,Flyer than everybody Section 8 just straight ...,"['', 'flyer', 'than', 'everybody', 'section', ...",flyer everybody section 8 straight cooked muh...,"(262144,[1303,1398,1745,3280,4372,4978,6715,82...","(262144,[1303,1398,1745,3280,4372,4978,6715,82...",en
6,6,Whoopty,CJ,34,Loyalty over royalty yall niggas know the vib...,"['', 'loyalty', 'over', 'royalty', 'yall', 'ni...",loyalty royalty yall niggas know vibes pxcoyo...,"(262144,[1689,3186,6116,6524,8538,10345,11209,...","(262144,[1689,3186,6116,6524,8538,10345,11209,...",en


In [68]:
# Remove songs that do not have english lyrics
for index, row in music_df.iterrows():
    if row['language'] != 'en':
        music_df = music_df.drop(index)
music_df.head()

,Unnamed: 0,song,artist,category,lyrics,words,filtered,hashedValues,features,language
0,0,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,34,You put me on a pedestal and tell me Im the b...,"['', 'you', 'put', 'me', 'on', 'a', 'pedestal'...",put pedestal tell im best raise sky im short ...,"(262144,[3386,3924,18184,21823,30548,31015,375...","(262144,[3386,3924,18184,21823,30548,31015,375...",en
1,1,positions,Ariana Grande,34,Heaven sent you to me Im just hopin I dont re...,"['', 'heaven', 'sent', 'you', 'to', 'me', 'im'...",heaven sent im hopin dont repeat history boy ...,"(262144,[7231,21823,31015,39504,51471,55334,64...","(262144,[7231,21823,31015,39504,51471,55334,64...",en
2,2,Therefore I Am,Billie Eilish,34,Im not your friend Or anything damn You think...,"['', 'im', 'not', 'your', 'friend', 'or', 'any...",im friend anything damn think youre man think...,"(262144,[18176,22346,23071,23087,24175,31015,3...","(262144,[18176,22346,23071,23087,24175,31015,3...",en
5,5,Errbody,Lil Baby,34,Flyer than everybody Section 8 just straight ...,"['', 'flyer', 'than', 'everybody', 'section', ...",flyer everybody section 8 straight cooked muh...,"(262144,[1303,1398,1745,3280,4372,4978,6715,82...","(262144,[1303,1398,1745,3280,4372,4978,6715,82...",en
6,6,Whoopty,CJ,34,Loyalty over royalty yall niggas know the vib...,"['', 'loyalty', 'over', 'royalty', 'yall', 'ni...",loyalty royalty yall niggas know vibes pxcoyo...,"(262144,[1689,3186,6116,6524,8538,10345,11209,...","(262144,[1689,3186,6116,6524,8538,10345,11209,...",en


In [69]:
# Update word list
words_list = []
for index, row in music_df.iterrows():
    filtered_words = music_df['filtered'][index]
    unique_words = list(set(filtered_words.strip().split(' ')))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
len(word_columns)

59444

In [74]:
# Create DataFrame for ML model
columns = ['song_name', 'artist_name']
columns.extend(word_columns)
columns.append('category/genre')
song_words_df = pd.DataFrame(columns=columns)
song_words_df['song_name'] = music_df['song']
song_words_df['artist_name'] = music_df['artist']
song_words_df['category/genre'] = music_df['category']
song_words_df = song_words_df.fillna(0)
song_words_df.head()

,song_name,artist_name,,south,triangular,shaun,inteprint,jamn,dayeleyves,myrioscope,...,puckered,industries,blisters,balacleivka,silliness,welders,영원,delilahs,okhorwan,category/genre
0,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34
1,positions,Ariana Grande,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34
2,Therefore I Am,Billie Eilish,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34
5,Errbody,Lil Baby,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34
6,Whoopty,CJ,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34


In [75]:
# Add the term frequencies to the DataFrame
for index, row in song_words_df.iterrows():
    for word in list(music_df.loc[index]['filtered'].strip().split(' ')):
        song_words_df.loc[index, word]+=1
song_words_df.head()

,song_name,artist_name,,south,triangular,shaun,inteprint,jamn,dayeleyves,myrioscope,...,puckered,industries,blisters,balacleivka,silliness,welders,영원,delilahs,okhorwan,category/genre
0,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34
1,positions,Ariana Grande,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34
2,Therefore I Am,Billie Eilish,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34
5,Errbody,Lil Baby,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34
6,Whoopty,CJ,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34


In [76]:
# Save DataFrame to CSV
song_words_df.to_csv('../preliminary_dataframes/lyric_TF.csv', index=False)

In [ ]:
# Make the song name be the index of the DataFrame